In [0]:
# Descritption: This program will detect if an email is spam(1) or not spam(0)
# CSCI 4741 002
# Final Project

#installing Libraries
#WordNinja is used to split strings into a list of words
!pip install wordninja

#Import Libraries
import numpy as np
import pandas as pd
import nltk
import string
import io
import wordninja

from nltk.corpus import stopwords
from google.colab import files 



#Using the upload feature provided by google colab we will promt the user to upload the email csv he/she would like tested
print("Please upload the emails you would like tested: ")
uploaded = files.upload()

#We will take the csv which was uploaded and turn it into a dataframe supported by pandas 
df = pd.read_csv(io.StringIO(uploaded['Emails.CSV'].decode('utf-8', 'ignore')))

#This will drop all columns who are entirely NA or "empty"
df=df.dropna(axis=1,how='all')

#Ask the user how he/she would like her emails filtered 
print("\n Please Pick a Filtration Method")
print("1: Filter Using A List Of Approved Contacts ")
print("2: Filter Using Our Spam Detection Process")
#Record the users response
filtration_method =input()
print()

#------------------------------Filter by Strict Address------------------------------
if (filtration_method == "1"):

  #Prompt the user to upload the csv of authorized contacts
  print("Please upload your authorized contacts list:" )
  safeContacts = files.upload()

  #Save the uploaded contacts list to a dataframe
  SafeContactsList = pd.read_csv(io.StringIO(safeContacts['Contacts.CSV'].decode('utf-8', 'ignore')))
  print()
  
  #We will now fliter all emails that have an approved contact.
  #A new column is also added title safe. (This is used to keep track of if an email is considered spam free or not)
  df["Safe"] = df["From: (Address)"].isin(SafeContactsList["(Address)"])

  #If the emails in the dataframe (df) DOES contain an address from the contacts list they will be given a (True) value under the newly added column titled (Safe)
  #If the emails in the dataframe (df) DOES NOT contain an address from the contacts list they will be given a (False) value under the newly added column titled (Safe)
  #After being labled they will be saved to either the NotSpam dataframe or Spam dataframe
  NotSpam = df.loc[df['Safe'] == True]
  Spam = df.loc[df['Safe'] == False]
  
  #Convert the NotSpam and Spam dataframes to a csv for future exporting
  Spam.to_csv('Possible_Spam_Emails.csv')
  NotSpam.to_csv('Safe_Emails.csv')

  #Tell the user that their emails are being filtered
  print("Spam Filtration Finished : Press Enter To Download Your Filtered Emails")

  #using the file download feature provided by google the user will be able to now download a csv that contains both the Spam and NotSpam csv files
  input()
  files.download("Safe_Emails.csv")
  files.download("Possible_Spam_Emails.csv")

#------------------------------Filter Spam Via Our Groups 3 Step Method------------------------------  
#Step 1: If the senders email address has any special characters, register as SPAM
#Step 2: If the senders email address has any random combination of letters before the @, register as SPAM.... (e.g. "info5T817fbc@fullofpositiveenergy.com")
#Step 3: If the email subject uses any special characters found often in spam email
if (filtration_method == "2"):
  
  #Creat a new column in our email.csv which we report if an email is spam
  df["Safe"] = True
  
  #Prompt the user to upload the csv of flagged keywords
  #print("Please upload your the flagged keywords/phrases:" )
  #FlaggedWords = files.upload()

  #**Save the uploaded words to a dataframe**
  #RedFlaggedWords = pd.read_csv(io.StringIO(FlaggedWords['RedFlags.CSV'].decode('utf-8', 'ignore')))
  #print()
  #****

  #Report and save how many emails we will be testing
  MailColumnSize = df.shape[0]

  #Through a forloop evaluate every email
  for x in range(MailColumnSize):

    #This reports how many words makeup an email address, a safe email usually uses less than 3
    WordSplitLength = 0

    #First we will be testing the actual address which is in position 3
    AddressTest = df.iat[x, 3]
    SubjectTest = df.iat[x, 0]

    #Only test the text before the @ symbol 
    AddressTest = AddressTest.split('@')[0]

    #This will see if said text has any special characters
    SpecialCharactersAddress = (AddressTest.isalnum())
    
    #This will see if the subject field has any commmon spam "characters"
    SpecialCharactersList = ["!","@","?","$","%","^","&","*","_","-","+","=","{","}","[","]","<",">",";",",",".","|","#"]
    SpecialCharactersSubjectTest = any(ele in SubjectTest for ele in SpecialCharactersList) 
  
    #If special characters are present in ADDRESS, register that email as SPAM
    if (SpecialCharactersAddress == False):
      df.loc[x, 'Safe'] = False

    #If special characters are present in SUBJECT, register that email as SPAM
    if (SpecialCharactersSubjectTest == True):
      df.loc[x, 'Safe'] = False

    #We will now save into WordSplitLength the number of words found before the @ symbol
    WordSplitLength = (len(wordninja.split(AddressTest)))

    #Following our filtration strategy, If an address has more than 3 words before the @ symbol, register as SPAM
    if (WordSplitLength > 3):
      df.loc[x, 'Safe'] = False

  #If the emails in the dataframe (df) DOES any of the following it will be given a (False) value under the newly added column titled (Safe)
  # ----------  If the senders email address has any random combination of letters before the @
  # ----------  If the senders email address has any special characters
  # ----------  If the email subject uses any special characters found often in spam email
  #If the emails in the dataframe (df) DOES not trigger the filters it will be given a (True) value under the newly added column titled (Safe)

  #After being labled they will be saved to either the NotSpam dataframe or Spam dataframe
  NotSpam = df.loc[df['Safe'] == True]
  Spam = df.loc[df['Safe'] == False]
  
  #Convert the NotSpam and Spam dataframes to a csv for future exporting
  Spam.to_csv('Possible_Spam_Emails.csv')
  NotSpam.to_csv('Safe_Emails.csv')

  #Tell the user that their emails are being filtered
  print("Spam Filtration Finished : Press Enter To Download Your Filtered Emails")

  #using the file download feature provided by google the user will be able to now download a csv that contains both the Spam and NotSpam csv files
  input()
  files.download("Safe_Emails.csv")
  files.download("Possible_Spam_Emails.csv")

    

  

Please upload the emails you would like tested: 


Saving Emails.CSV to Emails (1).CSV

 Please Pick a Filtration Method
1: Filter Using A List Of Approved Contacts 
2: Filter Using Our Spam Detection Process
2

Spam Filtration Finished : Press Enter To Download Your Filtered Emails

